In [9]:
from sklearn.experimental import enable_iterative_imputer
import pandas as pd
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
import kennard_stone as ks

In [10]:
PM25=pd.read_excel(r"../data/corectiong_123.xlsx");
PM25_raw=pd.read_excel(r"../data/pm25.xlsx");


In [11]:
PM25=pd.concat([PM25_raw,PM25.loc[:,['constrution_year', 'Area', 'floor_level', 'powerplant', 'road', 'motorway', 'forest', 'coastline',
       'airport', 'farmland', 'industrial', 'carpet', 'cookerhood', 'passive',
       'gastove', 'fireplace']]],axis=1);

In [12]:
PM25.date=pd.to_datetime(PM25.sadto)
PM25=PM25.assign(week=PM25.sadto.apply(lambda x: x.week))
PM25=PM25.assign(month=PM25.sadto.apply(lambda x: x.month))
PM25=PM25.assign(year=PM25.sadto.apply(lambda x: x.year))
PM25=PM25.assign(day=PM25.sadto.apply(lambda x: x.day_of_year))
PM25["corrected_week"]=PM25.day/7

/tmp/ipykernel_1163893/3369555380.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  PM25.date=pd.to_datetime(PM25.sadto)


In [13]:
PM25=PM25.loc[(PM25.pm_final.isna()==0)&(PM25.sv_final.isna()==0) ,:]
PM25=PM25.drop(['Unnamed: 0', 'fnr',"adrnum",'vejnvn', 'husn',"postnr","sadto"],axis=1);

In [16]:
PM25.isna().sum()

bnr                   0
flow                  0
pm_final              0
sv_final              0
maalnr                0
constrution_year    113
Area                164
floor_level           0
powerplant            0
road                  0
motorway              0
forest                0
coastline             0
airport               0
farmland              0
industrial            0
carpet               16
cookerhood           16
passive             120
gastove              16
fireplace            16
week                  0
month                 0
year                  0
day                   0
corrected_week        0
dtype: int64

In [17]:

scaler = StandardScaler()
imputer = IterativeImputer(random_state=0)
pm25=imputer.fit_transform(scaler.fit_transform(PM25))
pm25=scaler.inverse_transform(pm25)

In [18]:
PM25=pd.DataFrame(np.round(pm25,3),columns=PM25.columns,index=PM25.index)

In [21]:
PM25

,flow,pm_final,sv_final,maalnr,constrution_year,Area,floor_level,powerplant,road,motorway,...,carpet,cookerhood,passive,gastove,fireplace,week,month,year,day,corrected_week
bnr,,,,,,,,,,,,,,,,,,,,,
1.0,4.07,3.748,0.162,1.0,1979.0,126.0,0.0,0.000,11222.958,1036.640,...,0.0,365.0,365.000,365.0,0.0,27.0,7.0,2000.0,187.0,26.714
1.0,3.75,13.851,3.060,2.0,1979.0,126.0,0.0,0.000,11222.958,1036.640,...,0.0,365.0,365.000,365.0,1.0,50.0,12.0,2000.0,347.0,49.571
1.0,4.22,6.938,0.343,3.0,1979.0,126.0,0.0,0.000,11222.958,1036.640,...,0.0,365.0,365.000,365.0,1.0,28.0,7.0,2001.0,195.0,27.857
3.0,3.78,83.668,0.962,1.0,1924.0,74.0,1.0,4948.337,21995.079,1176.864,...,365.0,0.0,627.067,365.0,0.0,27.0,7.0,2000.0,187.0,26.714
3.0,3.74,59.950,1.790,2.0,1924.0,74.0,1.0,4948.337,21995.079,1176.864,...,30.0,0.0,439.641,365.0,0.0,50.0,12.0,2000.0,347.0,49.571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450.0,3.96,13.040,0.562,3.0,1968.0,94.0,1.0,0.000,20693.860,3318.698,...,0.0,104.0,730.000,0.0,0.0,22.0,5.0,2003.0,147.0,21.000
451.0,3.88,23.291,1.544,1.0,1987.0,95.0,0.0,0.000,20693.860,3318.698,...,0.0,365.0,0.000,180.0,100.0,14.0,4.0,2002.0,95.0,13.571
452.0,3.92,18.355,1.033,1.0,1925.0,79.0,1.0,46.576,16419.647,9154.441,...,0.0,0.0,138.994,300.0,0.0,10.0,3.0,2002.0,69.0,9.857


In [24]:
bnr=PM25.reset_index().loc[:,["bnr","Area","passive","flow"]].groupby("bnr").mean().reset_index()
X_train, X_test, y_train, y_test = ks.train_test_split(bnr.loc[:,["Area","passive","flow"]], bnr.bnr, test_size = 0.2);

train=PM25.loc[y_train.values,:]
test=PM25.loc[y_test.values,:]

/data/michaelf/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-04-02 09:46:42,797 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.

/data/michaelf/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-04-02 09:46:42,798 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.

/data/michaelf/miniconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [30]:
PM25.loc[train.index,"label"]=0
PM25.loc[test.index,"label"]=1

In [32]:
PM25.to_excel("../data/Pre_flow_cal.xlsx")